In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
import pandas as pd
import cv2
import numpy as np
from sklearn.metrics import  classification_report
import features as features
import utilities as util

In [ ]:
'''
parms: train data (X_train)
       test data  (X_test)
       train classes  (y_train)

----------------------------

returns: test classes  (y_test)
'''


def svm_classify(X_train,y_train,X_test, kernel='rbf'):
    clf = svm.SVC(kernel=kernel)
    clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
    return clf

In [ ]:
'''
parms: train data (X_train)
       test data  (X_test)
       train classes  (y_train)

----------------------------

returns: test classes  (y_test)
'''

def random_forest_classify(X_train,y_train,X_test, max_depth=2, n_estimators=50):
    clf = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
    clf.fit(X_train, y_train)
#     y_pred=clf.predict(X_test)
    return clf

In [ ]:
'''
parms: train data (X_train)
       test data  (X_test)
       train classes  (y_train)

----------------------------

returns: test classes  (y_test)
'''

def knn_classify(X_train,y_train,X_test, n_neighbors=5):
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors)
        neigh.fit(X_train, y_train)
       #  y_pred=neigh.predict(X_test)
        return neigh

In [ ]:
'''
parms: test classes of models tarined by  --> 1) svm
                                              2) random_forest
                                              3)knn

----------------------------

returns: test classes after taking the vote of the 3 classes
'''

def voting_system(y_pred_svm,y_pred_random_forest,y_pred_knn,y_test):
    y_pred=np.add(np.add(y_pred_knn,y_pred_svm),y_pred_random_forest)
    y_pred[y_pred <=1]=0
    y_pred[y_pred >1]=1  
    return y_pred


In [ ]:
def apply_PCA(X,n_components=5, verbose=False):
    sc = StandardScaler()
    X_copy = np.copy(X)
    X_copy = sc.fit_transform(X_copy)
    pca = PCA(n_components = n_components)
    X_copy = pca.fit_transform(X_copy)
    
    if(verbose):
        print("preserved vairance: ", np.sum(pca.explained_variance_ratio_))
    
    return X_copy

In [ ]:
def pre_process(image):
    thresh = util.threshold_image(image)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    return contours

In [ ]:
def extract_hinge(contours):
    # extract features
    hinge_feature = features.get_hinge_features(contours)

    return hinge_feature

In [ ]:
def extract_cold(contours):
    cold_feature = features.get_cold_features(contours)
    
    return cold_feature

In [ ]:
def extract_hog(image):
    hog_feature = features.extract_hog_features(image)
    return hog_feature
    